In [1]:
rm(list = ls())

library(dplyr)
library(ggplot2)
library(readr)
library(tidyverse)
library(viridis)
library(ggmap)
library(sf)
library(data.table)
library(bit64)
library(corrplot)

setwd("C:/Users/momaleki/Desktop/Project")
getwd()

df_No_SCI <- fread("Data/ToMerge/df_No_SCI.csv")
df_No_SCI <- df_No_SCI[, -c(1)]
df <- df_No_SCI
df <- df[, -c(8:9, 11:14, 26:28 )]


colnames(df) <- c( 'NEVER', 'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS', 
                   'FIPS', 'Combined','State', 'Percent_Less_HS', 'Percent_HS', 'Percent_Less_College', 'Percent_Higher_College', 
                 'Trump_Win', 'Biden_Win', 'LAT', 'Lon', 'Unemployment', 'Movement_Change', 'Percent_InTouch', 
                 'Percent_Rural', 'Pop_Density', 'Poverty', 'COVID_Confirmed', 'COVID_Death', 'Percent_Ess_Workers', 
                 'Hospital_AllBeds', 'Hospital_Occ_rate', 'COVID_TestCap', 'COVID_Testing', 'COVID_Clinic', 
                 'Vaccine', 'Crime')


df <- df[, -c(14:15)]
head(df[, c(1:15)])


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v tibble  2.1.1     v purrr   0.3.2
v tidyr   0.8.3     v stringr 1.4.0
v tibble  2.1.1     v forcats 0.4.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Warning message:
"package 'viridis' was built under R version 3.6.3"Loading required package: viridisLite
Warning message:
"package 'viridisLite' was built under R version 3.6.3"Warning message:
"package 'ggmap' was buil

[1] "C:/Users/momaleki/Desktop/Project"

NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,FIPS,Combined,State,Percent_Less_HS,Percent_HS,Percent_Less_College,Percent_Higher_College,Trump_Win,Lon,Unemployment
0.026,0.006,0.106,0.051,0.811,10001,4.615,Delaware,0.126,0.329,0.308,0.237,0,-75.50298,4.2
0.026,0.006,0.106,0.051,0.811,10001,4.615,Delaware,0.126,0.329,0.308,0.237,0,-75.50298,4.2
0.026,0.006,0.106,0.051,0.811,10001,4.615,Delaware,0.126,0.329,0.308,0.237,0,-75.50298,4.2
0.026,0.006,0.106,0.051,0.811,10001,4.615,Delaware,0.126,0.329,0.308,0.237,0,-75.50298,4.2
0.027,0.013,0.041,0.133,0.786,10003,4.638,Delaware,0.084,0.308,0.246,0.362,0,-75.64413,3.7
0.027,0.013,0.041,0.133,0.786,10003,4.638,Delaware,0.084,0.308,0.246,0.362,0,-75.64413,3.7


## Mask Usage

In [12]:
df <- fread("Data/mask-use-by-county.csv")
#mydata <- df[, c(1:6)] %>% 
        #        group_by(FIPS) %>%
             #       summarise_all(mean)

head(df)

COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
1001,0.053,0.074,0.134,0.295,0.444
1003,0.083,0.059,0.098,0.323,0.436
1005,0.067,0.121,0.120,0.201,0.491
1007,0.020,0.034,0.096,0.278,0.572
1009,0.053,0.114,0.180,0.194,0.459
1011,0.031,0.040,0.144,0.286,0.500


In [9]:
test <- data.frame(matrix(nrow = nrow(mydata), ncol = 5))
model <- kmeans(mydata$ALWAYS, 5)
test[, c(1)] <- mydata$FIPS
test[, c(2)] <- model$cluster


model <- kmeans(mydata$ALWAYS, 4)
test[, c(1)] <- mydata$FIPS
test[, c(3)] <- model$cluster


model <- kmeans(mydata$ALWAYS, 3)
test[, c(1)] <- mydata$FIPS
test[, c(4)] <- model$cluster


model <- kmeans(mydata$ALWAYS, 2)
test[, c(1)] <- mydata$FIPS
test[, c(5)] <- model$cluster

colnames(test) <- c("FIPS", "Cluster = 6", "Cluster = 6", "Cluster = 6", "Cluster = 6")




head(test)
write.csv(test, 'Data/Clustering/Mask_Use.csv', fileEncoding="UTF-8")
test <- NULL

FIPS,Cluster = 6,Cluster = 6,Cluster = 6,Cluster = 6
10001,4,3,3,2
10003,4,3,3,2
10005,4,3,3,2
12001,4,3,3,2
12005,3,4,1,1
12007,2,1,2,1


## Election Decisions 

In [21]:
Election_decisions <- fread("Data/election decisions by fips country.csv")

#  Election Decisions 
colnames(Election_decisions) <- paste("Election", colnames(Election_decisions), sep = "_")
Election_decisions$FIPS111 <- as.double(Election_decisions$Election_id)
Election_decisions <- Election_decisions[, -c(2, 3, 4, 5, 11, 12 )]
Election_decisions$Election_Trump <- ifelse(Election_decisions$Election_lname == "Trump", 1, 0)
Election_decisions$Election_Biden <- ifelse(Election_decisions$Election_lname == "Biden", 1, 0)
Election_decisions$Trump_Votes <- Election_decisions$Election_votes * Election_decisions$Election_Trump
Election_decisions$Biden_Votes <- Election_decisions$Election_votes * Election_decisions$Election_Biden

Election_decisions_1 <- Election_decisions %>% 
                            filter(Election_race == "President") %>%
                            group_by(FIPS111) %>%
                                summarize(Winner_Votes = max(Election_votes), 
                                         Trump_Votes = max(Trump_Votes), 
                                         Biden_Votes = max(Biden_Votes))
Election_decisions_1$Trump_Win <- ifelse(Election_decisions_1$Trump_Votes > Election_decisions_1$Biden_Votes, 
                                  1, 0)
Election_decisions_1$Biden_Win <- ifelse(Election_decisions_1$Trump_Votes < Election_decisions_1$Biden_Votes, 
                                  1, 0)
Election_decisions_1 <- subset(Election_decisions_1, floor(log10(FIPS111))  == 4 | floor(log10(FIPS111))  == 3)
Election_decisions_1 <- Election_decisions_1[-c(2:4)]
Election_decisions <- Election_decisions_1

head(Election_decisions)

write.csv(Election_decisions, 'Data/Clustering/Election.csv', fileEncoding="UTF-8")

FIPS111,Trump_Win,Biden_Win
1001,1,0
1003,1,0
1005,1,0
1007,1,0
1009,1,0
1011,0,1


## Population Density

In [24]:
Population_density <- fread("Data/Population_density.csv")

# Population Density
colnames(Population_density) <- paste("Pop_Density", colnames(Population_density), sep = "_")
Population_density$FIPS <- as.double(Population_density$Pop_Density_GEOID)
Population_density$Population_Density <- as.double(Population_density$Pop_Density_B01001_calc_PopDensity)
Population_density <- Population_density[, -c(1:23)]

write.csv(Population_density, 'Data/Clustering/Pop_density.csv', fileEncoding="UTF-8")


In [23]:
test <- data.frame(matrix(nrow = nrow(mydata), ncol = 5))

model <- kmeans(mydata$Population_Density, 5)
test[, c(1)] <- mydata$FIPS
test[, c(2)] <- model$cluster


model <- kmeans(mydata$Population_Density, 4)
test[, c(1)] <- mydata$FIPS
test[, c(3)] <- model$cluster


model <- kmeans(mydata$Population_Density, 3)
test[, c(1)] <- mydata$FIPS
test[, c(4)] <- model$cluster


model <- kmeans(mydata$Population_Density, 2)
test[, c(1)] <- mydata$FIPS
test[, c(5)] <- model$cluster

colnames(test) <- c("FIPS", "Cluster = 5", "Cluster = 4", "Cluster = 3", "Cluster = 2")

head(test)
write.csv(test, 'Data/Clustering/Pop_density.csv', fileEncoding="UTF-8")

FIPS,Cluster = 5,Cluster = 4,Cluster = 3,Cluster = 2
1001,2,3,1,1
1003,2,3,1,1
1005,2,3,1,1
1007,2,3,1,1
1009,2,3,1,1
1011,2,3,1,1


## Movement Range

In [26]:
movement_range <- fread("Data/movement-range-2021-05-23.txt")

# Movement Range
movement_range$date <- movement_range$ds
movement_range$ds <- NULL
movement_range$country_code <- movement_range$country
movement_range$country <- NULL
movement_range$polygon_source <- NULL
movement_range$polygon_name <- NULL
movement_range$change_in_movement <- movement_range$all_day_bing_tiles_visited_relative_change
movement_range$all_day_bing_tiles_visited_relative_change <- NULL
movement_range$proportion_intouch_users <- movement_range$all_day_ratio_single_tile_users
movement_range$all_day_ratio_single_tile_users <- NULL
movement_range$baseline_calc_time <- movement_range$baseline_name
movement_range$baseline_name <- NULL
movement_range$baseline_calc_how <- movement_range$baseline_type
movement_range$baseline_type <- NULL
movement_range = filter(movement_range, movement_range$country_code == "USA")
movement_range$date <- as.Date(movement_range$date, format = "%Y-%m-%d")
movement_range = subset(movement_range, date > "2020-07-1" & date < "2020-07-15")
colnames(movement_range) <- paste("Movement_Range", colnames(movement_range), sep = "_")
movement_range$FIPS <- as.double(movement_range$Movement_Range_polygon_id)
movement_range$Movement_Range_polygon_id <- NULL
movement_range <- movement_range[, -c(2, 5, 6)]

movement_range <- movement_range %>% 
                    group_by(FIPS) %>%
                            summarize(Change_in_Movement = mean(Movement_Range_change_in_movement),
                                      Intouch_users_Proportion = mean(Movement_Range_proportion_intouch_users))

mydata <- movement_range
write.csv(mydata, 'Data/Clustering/movement.csv', fileEncoding="UTF-8")


In [120]:
test <- data.frame(matrix(nrow = nrow(mydata), ncol = 5))

model <- kmeans(mydata$Change_in_Movement, 5)
test[, c(1)] <- mydata$FIPS
test[, c(2)] <- model$cluster


model <- kmeans(mydata$Change_in_Movement, 4)
test[, c(1)] <- mydata$FIPS
test[, c(3)] <- model$cluster


model <- kmeans(mydata$Change_in_Movement,3)
test[, c(1)] <- mydata$FIPS
test[, c(4)] <- model$cluster


model <- kmeans(mydata$Change_in_Movement, 2)
test[, c(1)] <- mydata$FIPS
test[, c(5)] <- model$cluster

colnames(test) <- c("FIPS", "Cluster = 5", "Cluster = 4", "Cluster = 3", "Cluster = 2")

head(test)
write.csv(test, 'Data/Clustering/movement.csv', fileEncoding="UTF-8")

FIPS,Cluster = 5,Cluster = 4,Cluster = 3,Cluster = 2
1001,5,2,1,1
1003,4,3,2,2
1005,5,2,1,1
1007,5,2,1,1
1009,5,2,1,1
1013,5,2,1,1


## Education 

In [137]:
Education_level <- fread("Data/Education level by Country 1970 - 2019.csv")

# Education Level
colnames(Education_level) <- paste("Education_Levels", colnames(Education_level), sep = "_")
Education_level$FIPS <- as.double(Education_level$`Education_Levels_FIPS Code`)
Education_level$`Education_Levels_FIPS Code` <- NULL
Education_level <- Education_level[, -c(0:38)]
Education_level <- subset(Education_level, floor(log10(FIPS))  == 4)
                    
colnames(Education_level) <- c('Count_Less_HighSchool', 'Count_HighSchool', 'Count_Less_College', 
                              'Count_Higher_College', 'Percent_Less_HighSchool', 'Percent_HighSchool', 
                             'Percent_Less_College', 'Percent_Higher_College', 'FIPS')

Education_level$Percent_Less_HighSchool <- Education_level$Percent_Less_HighSchool/100
Education_level$Percent_HighSchool <- Education_level$Percent_HighSchool/100
Education_level$Percent_Less_College <- Education_level$Percent_Less_College/100
Education_level$Percent_Higher_College <- Education_level$Percent_Higher_College/100

head(Education_level)

mydata <- Education_level[, c(8, 9)]
mydata <- mydata[complete.cases(mydata), ]

Count_Less_HighSchool,Count_HighSchool,Count_Less_College,Count_Higher_College,Percent_Less_HighSchool,Percent_HighSchool,Percent_Less_College,Percent_Higher_College,FIPS
"66,816","209,449","178,917","214,138",0.100,0.313,0.267,0.320,10000
"14,834","38,843","36,347","28,030",0.126,0.329,0.308,0.237,10001
"32,032","118,233","94,434","138,627",0.084,0.308,0.246,0.362,10003
"19,950","52,373","48,136","47,481",0.119,0.312,0.287,0.283,10005
"44,850","83,185","76,822","289,259",0.091,0.168,0.155,0.585,11000
"44,850","83,185","76,822","289,259",0.091,0.168,0.155,0.585,11001


In [139]:
test <- data.frame(matrix(nrow = nrow(mydata), ncol = 5))

model <- kmeans(mydata$Percent_Higher_College, 5)
test[, c(1)] <- mydata$FIPS
test[, c(2)] <- model$cluster


model <- kmeans(mydata$Percent_Higher_College, 4)
test[, c(1)] <- mydata$FIPS
test[, c(3)] <- model$cluster


model <- kmeans(mydata$Percent_Higher_College, 3)
test[, c(1)] <- mydata$FIPS
test[, c(4)] <- model$cluster


model <- kmeans(mydata$Percent_Higher_College, 2)
test[, c(1)] <- mydata$FIPS
test[, c(5)] <- model$cluster

colnames(test) <- c("FIPS", "Cluster = 5", "Cluster = 4", "Cluster = 3", "Cluster = 2")

head(test)
write.csv(test, 'Data/Clustering/Education.csv', fileEncoding="UTF-8")

FIPS,Cluster = 5,Cluster = 4,Cluster = 3,Cluster = 2
10000,4,2,3,2
10001,3,4,3,1
10003,4,2,2,2
10005,3,2,3,2
11000,1,3,2,2
11001,1,3,2,2


## Vaccination 

In [27]:
Vaccine <- fread("Data/COVID_Data/data/COVID-19_Vaccinations_in_the_United_States_County.csv")

# Vaccination 
Vaccine <- Vaccine %>%
                  group_by(FIPS) %>%
                            summarize(Vaccine_completeness_pct = mean(Series_Complete_Pop_Pct))

Vaccine$FIPS <- as.double(Vaccine$FIPS)
Vaccine$Vaccine_completeness_pct <- Vaccine$Vaccine_completeness_pct / 100


head(Vaccine)

mydata <- Vaccine[complete.cases(Vaccine), ]

write.csv(mydata, 'Data/Clustering/Vaccination.csv', fileEncoding="UTF-8")

Warning message in eval(expr, envir, enclos):
"NAs introduced by coercion"

FIPS,Vaccine_completeness_pct
1001,0.08196067
1003,0.11169101
1005,0.08088764
1007,0.07486517
1009,0.06486517
1011,0.12137079


In [143]:
test <- data.frame(matrix(nrow = nrow(mydata), ncol = 5))

model <- kmeans(mydata$Vaccine_completeness_pct, 5)
test[, c(1)] <- mydata$FIPS
test[, c(2)] <- model$cluster


model <- kmeans(mydata$Vaccine_completeness_pct, 4)
test[, c(1)] <- mydata$FIPS
test[, c(3)] <- model$cluster


model <- kmeans(mydata$Vaccine_completeness_pct, 3)
test[, c(1)] <- mydata$FIPS
test[, c(4)] <- model$cluster


model <- kmeans(mydata$Vaccine_completeness_pct, 2)
test[, c(1)] <- mydata$FIPS
test[, c(5)] <- model$cluster

colnames(test) <- c("FIPS", "Cluster = 5", "Cluster = 4", "Cluster = 3", "Cluster = 2")

head(test)
write.csv(test, 'Data/Clustering/Vaccination.csv', fileEncoding="UTF-8")

FIPS,Cluster = 5,Cluster = 4,Cluster = 3,Cluster = 2
1001,5,4,1,1
1003,5,4,3,1
1005,5,4,1,1
1007,5,4,1,1
1009,5,1,1,1
1011,5,4,3,1
